In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_palette("GnBu_d")
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime
from datetime import timedelta
import math
import random 
import timeit
from haversine import haversine # conda install -c conda-forge haversine ''

In [2]:
BRM = pd.read_csv("resources/bremen.csv")
FRB = pd.read_csv("resources/freiburg.csv")

In [3]:
BRM["trip_duration"] = BRM["trip_duration"].astype(str).str.extract("days (.*?)\.")
FRB["trip_duration"] = FRB["trip_duration"].astype(str).str.extract("days (.*?)\.")

To summarize and work with the duration of the Trips we need to coinvert them to a useful type:

In [4]:
BRM["trip_duration"] = BRM["trip_duration"].apply(pd.Timedelta)
FRB["trip_duration"] = FRB["trip_duration"].apply(pd.Timedelta)

After the convertion we sum the time daily driven:

In [5]:
BRM_daily_driven = BRM.groupby("day").agg({"trip_duration":"sum"})
FRB_daily_driven = FRB.groupby("day").agg({"trip_duration":"sum"})

Morgens: 6 - 11 Uhr | Mittags: 11 - 16 | Abends: 16 - 21 | Nachts: 21 - 6

In [6]:
BRM["daytime"] = "null"
FRB["daytime"] = "null"

In [7]:
BRM_morning = BRM[(BRM["time"] > "06:00:00") & (BRM["time"] <= "11:00:00")]
BRM_noon = BRM[(BRM["time"] > "11:00:00") & (BRM["time"] <= "16:00:00")]
BRM_afternoon = BRM[(BRM["time"] > "16:00:00") & (BRM["time"] <= "21:00:00")]
BRM_night = BRM[(BRM["time"] > "21:00:00")] + BRM[(BRM["time"] <= "06:00:00")]

In [8]:
FRB_morning = FRB[(FRB["time"] > "06:00:00") & (FRB["time"] <= "11:00:00")]
FRB_noon = FRB[(FRB["time"] > "11:00:00") & (FRB["time"] <= "16:00:00")]
FRB_afternoon = FRB[(FRB["time"] > "16:00:00") & (FRB["time"] <= "21:00:00")]
FRB_night = FRB[(FRB["time"] > "21:00:00")] + FRB[(FRB["time"] <= "06:00:00")]

In [9]:
for i in BRM_morning.index:
   BRM.at[i , "daytime"] = 0

for i in BRM_noon.index:
   BRM.at[i , "daytime"] = 1

for i in BRM_afternoon.index:
   BRM.at[i , "daytime"] = 2

for i in BRM_night.index:
   BRM.at[i , "daytime"] = 3

In [10]:
for i in FRB_morning.index:
   FRB.at[i , "daytime"] = 0

for i in FRB_noon.index:
   FRB.at[i , "daytime"] = 1

for i in FRB_afternoon.index:
   FRB.at[i , "daytime"] = 2

for i in FRB_night.index:
   FRB.at[i , "daytime"] = 3

In [11]:
BRM_daytime_driven = BRM.groupby(["day","daytime"]).agg({"trip_duration":"sum"})
BRM_daytime_driven["trip_duration"] = BRM_daytime_driven["trip_duration"].astype(str).str.extract("days (.*?)\.")
BRM_daytime_driven 

trip_duration
day        daytime              
2019-01-20 0            04:21:00
           1            20:37:00
           2            08:41:00
           3            05:38:00
2019-01-21 0            16:45:00
...                          ...
2020-01-19 3            19:32:00
2020-01-20 0            21:32:00
           1            21:54:00
           2            17:04:00
           3            06:35:00

[1451 rows x 1 columns]

In [12]:
FRB_daytime_driven = FRB.groupby(["day","daytime"]).agg({"trip_duration":"sum"})
FRB_daytime_driven["trip_duration"] = FRB_daytime_driven["trip_duration"].astype(str).str.extract("days (.*?)\.")
FRB_daytime_driven 

trip_duration
day        daytime              
2019-05-06 1            00:07:00
2019-05-07 0            00:07:00
           1            00:32:00
2019-05-08 0            00:03:00
           1            00:18:00
...                          ...
2020-01-19 3            02:26:00
2020-01-20 0            05:46:00
           1            22:31:00
           2            14:34:00
           3            12:07:00

[1008 rows x 1 columns]

In [13]:
FRB_prove = FRB[FRB["day"] == "2019-05-07"]
FRB_prove

,day,time,b_number,city,trip_duration,orig_lat,orig_lng,dest_lat,dest_lng,daytime
1,2019-05-07,10:42:00,32560,freiburg,00:07:00,47.994191,7.796853,47.991960,7.797405,0
2,2019-05-07,11:02:00,32560,freiburg,00:09:00,47.992044,7.797352,47.992000,7.797478,1
3,2019-05-07,12:48:00,32560,freiburg,00:23:00,47.992107,7.797499,47.994271,7.796668,1
